In [11]:
import pandas as pd
import unicodedata
import re
import os

import datasets
from datasets import list_datasets, load_dataset
from transformers import AutoTokenizer
from pprint import pprint

In [12]:
def _setHFToken():
    with open("../hf_token.txt", "r") as file:
        token = file.read()
        
    return token
os.environ['HF_TOKEN'] = _setHFToken()
# os.environ['HF_TOKEN']

In [13]:
dataset = load_dataset("jerryjalapeno/nart-100k-synthetic", split="train", token=os.environ['HF_TOKEN'])

In [ ]:
import pathlib 
import shutil

home_dir = pathlib.Path.home()
dataset_dir = home_dir / ".cache" / "huggingface" / "datasets"
    
shutil.rmtree(str(dataset_dir))

In [14]:
pprint(dataset.info.__dict__)

{'builder_name': 'json',
 'citation': '',
 'config_name': 'default',
 'dataset_name': 'nart-100k-synthetic',
 'dataset_size': 416296694,
 'description': '',
 'download_checksums': {'hf://datasets/jerryjalapeno/nart-100k-synthetic@e10b686e0666b44c4807571ec6d47a230cf62256/vicunaformatfixedfinal.json': {'checksum': None,
                                                                                                                                                 'num_bytes': 546475257}},
 'download_size': 546475257,
 'features': {'conversations': [{'from': Value(dtype='string', id=None),
                                 'value': Value(dtype='string', id=None)}],
              'id': Value(dtype='string', id=None)},
 'homepage': '',
 'license': '',
 'post_processed': None,
 'post_processing_size': None,
 'size_in_bytes': 962771951,
 'splits': {'train': SplitInfo(name='train',
                               num_bytes=416296694,
                               num_examples=99086,
            

In [15]:
temp = dataset[0]
temp = temp['conversations'][0]
temp = temp['value']
print(type(temp))
print(temp)

<class 'str'>
I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me.


In [16]:
SYSTEM_PROMPT = """You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""
def preprocessText(text):
  text = re.sub(r'Alex', '', text)
  text = re.sub(r'Charlie', '', text)
  text = text.lower().strip()
  # remove ", " when it appears at the start of a sentence
  text = re.sub(r'^, ', '', text)
  # remove " ." with "."
  text = re.sub(r' \.', '.', text)
  # remove " ," with ","
  text = re.sub(r' ,', ',', text)
  # remove " ?" with "?"
  text = re.sub(r' \?', '?', text)
  # remove " !" with "!"
  text = re.sub(r' \!', '!', text)
  # remove ",." with "."
  text = re.sub(r',\.', '.', text)
  # remove ",?" with "?"
  text = re.sub(r',\?', '?', text)
  # remove more than one space
  text = re.sub(r' +', ' ', text)

  restext = ""
  for ch in unicodedata.normalize('NFD', text):
    if unicodedata.category(ch) != 'Mn':
      restext+=ch

  # restext = re.sub(r"([.!?])", r" \1", restext)
  restext = re.sub(r"[^a-zA-Z.!?]+", r" ", restext)

  short_forms = {
        r"\bi ve\b": "i have",
        r"\bi m\b": "i am",
        r"\bisn t\b": "is not",
        r"\baren t\b": "are not",
        r"\bwasn t\b": "was not",
        r"\bweren t\b": "were not",
        r"\bhaven t\b": "have not",
        r"\bhasn t\b": "has not",
        r"\bhadn t\b": "had not",
        r"\bwon t\b": "will not",
        r"\bwouldn t\b": "would not",
        r"\bdon t\b": "do not",
        r"\bdoesn t\b": "does not",
        r"\bdidn t\b": "did not",
        r"\bcan t\b": "cannot",
        r"\bcouldn t\b": "could not",
        r"\bshouldn t\b": "should not",
        r"\bmightn t\b": "might not",
        r"\bmustn t\b": "must not",
        r"\bain t\b": "am not"
    }

  for short_form, full_form in short_forms.items():
      restext = re.sub(short_form, full_form, restext)

  return restext.strip()

print(preprocessText(temp))

def preprocessDataset(row):
  id = row['id']
  row = row['conversations']
  for conversation in row:
    if conversation.get('from') == 'human':
      conversation['role'] = "human"
    elif conversation.get('from') == 'gpt':
      conversation['role'] = "assistant"

    conversation['content'] = preprocessText(conversation.get('value'))
    del conversation['from']
    del conversation['value']

  sys_dict = {
        'role': "system",
        'content': SYSTEM_PROMPT
    }
  row.insert(0, sys_dict)
  # Conversational format: messages
  return {"messages":row}

i have been feeling so sad and overwhelmed lately. work has become such a massive source of stress for me.


In [17]:
dataset = dataset.map(preprocessDataset, remove_columns=['conversations'])

Map: 100%|██████████| 99086/99086 [02:38<00:00, 623.39 examples/s]


In [21]:
tokenizer = AutoTokenizer.from_pretrained("philschmid/gemma-tokenizer-chatml")
tokenizer.padding_side = 'right' 
original_tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

# get special tokens
print(tokenizer.special_tokens_map)
print(original_tokenizer.special_tokens_map)

assert len(tokenizer) == len(original_tokenizer), "tokenizer are not having the same length"

{'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}
{'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<start_of_turn>', '<end_of_turn>']}


In [19]:
# remove conversation with more than 1024 tokens, for training memory reasons.
dataset = dataset.map(lambda x: {"input_ids_length": len(tokenizer.apply_chat_template(x["messages"]))})
# filter out the samples that are too long
max_input_length = 1024
dataset = dataset.filter(lambda x: x["input_ids_length"] <= max_input_length)
dataset = dataset.remove_columns(["input_ids_length"])
print(dataset)

Filter: 100%|██████████| 99086/99086 [00:07<00:00, 13209.96 examples/s]

Dataset({
    features: ['id', 'messages'],
    num_rows: 71878
})


In [20]:
dataset.save_to_disk('../dataset')

Saving the dataset (1/1 shards): 100%|██████████| 71878/71878 [00:00<00:00, 124553.06 examples/s]
